In [ ]:
from playwright.async_api import async_playwright
# from undetected_playwright.async_api import async_playwright
import time
# from PIL import Image
import asyncio
import subprocess
import re

In [ ]:
async def open_browser(page):
    # await page.set_viewport_size({'width': 1920, 'height': 1080})
    # await page.set_viewport_size({'width': 1600, 'height': 900})
    await page.emulate_media(color_scheme='dark')
    # weblink="https://www.iaai.com/VehicleDetail/39554640~US"
    # weblink="https://www.iaai.com"
    weblink="https://www.copart.com/login/"
    await page.goto(weblink, wait_until='load')
    return page

In [ ]:
playwright = await async_playwright().start()
args = []
# disable navigator.webdriver:true flag
# args.append("--disable-blink-features=AutomationControlled")

# browser = await playwright.chromium.launch(args=args,headless=False)
browser = await playwright.chromium.launch(headless=False)
# browser = await playwright.firefox.launch(headless=False)
# browser = playwright.firefox.launch(headless=False)
context = await browser.new_context()
page = await context.new_page()
browse =await open_browser(page=page)
# Waiting for the page to load
time.sleep(5)

In [ ]:
# Find the email input field by its ID
email_input = await page.query_selector('#username')

# Enter the desired content
email = "matti19913@gmail.com"
await email_input.fill(email)

password_input = await page.query_selector('#password')

# Clear the existing value (if any)
await password_input.fill('')

# Enter the desired content
password = "Copart2023!"
await password_input.fill(password)

# Clicking on the remember me checkbox
await page.click('text=Remember?')

# Clicking on the login button
await page.click('text=Sign Into Your Account')
# Waiting for the page to load
await asyncio.sleep(5)

In [ ]:
# await page.click('button#onetrust-accept-btn-handler')

In [ ]:
import pymongo
import os
from dotenv import load_dotenv
load_dotenv()

client = pymongo.MongoClient(os.getenv("MONGO_URI"))
db = client['Copart']
collection = db['AuctionLinks']
# collection.delete_many({})

count=0
opened_links = 0


In [ ]:
# await page.goto("https://www.copart.com/auctionDashboard")
# await page.goto("https://www.copart.com/todaysAuction")

In [ ]:
# all_auctions=await page.query_selector_all('a.btn.btn-green.joinsearch.small')
# print(all_auctions)
# print(f"Total Auctions: {len(all_auctions)}")

In [ ]:
# print(f"Total Auctions: {len(all_auctions)}")
# while all_auctions:
#     i=all_auctions.pop()
#     link_element = await i.get_attribute("href")
#     relative_url = link_element.lstrip('.')
#     whole_url = "https://www.copart.com"+relative_url
#     if collection.find_one({'link': whole_url}) is None:
#         print(whole_url)
#         # Upload the link to MongoDB
#         collection.insert_one({'link': whole_url, 'Info': "None"})

In [ ]:
cursor = collection.find({"Info": "None"})
instances={}
elements={}
opened_links = 0
data={}

In [ ]:
# while opened_links<10:
#     cursor = collection.find_one_and_update({"Info": "None"}, {"$set": {"Info": "Processing"}})
#     auction_link = cursor['link']
#     print(auction_link)
#     await page.goto(auction_link)

In [ ]:
cursor = collection.find_one_and_update({"Info": "None"}, {"$set": {"Info": "Processing"}})
auction_link = cursor['link']
print(auction_link)
await page.goto(auction_link)

In [ ]:
iframe_element=await page.query_selector('div.auction5iframe')
iframe=await iframe_element.query_selector("iframe")
content = await iframe.content_frame()

In [ ]:

await asyncio.sleep(5)
while len(await content.query_selector_all('gridster-item.ng-star-inserted'))<10:
    add_auction=await content.wait_for_selector('span.nav-option-on.addauctionbtn')
    await asyncio.sleep(2)
    await add_auction.click()
    all_auctions=await content.query_selector_all('text.ng-star-inserted')
# await content.click('span.nav-option-on')

In [ ]:
baseauction_url="https://www.copart.com/auctionDashboard?auctionDetails="

In [ ]:
# openeing the auction link
# checking auction sections
all_auctions=await content.query_selector_all('gridster-item.ng-star-inserted')
# len(all_auctions)

In [ ]:
# For managing the auction
collected_auctions=set()
for check in all_auctions:
    if await check.query_selector('tr.liveAuction.ng-star-inserted'):
        # checking the first join button
        join_button=await check.query_selector('tr.liveAuction.ng-star-inserted')
        # checking the auction
        locale=await check.query_selector('div.yardName-MACRO')
        auction_locale=await locale.get_attribute('title')
        auction_locale=auction_locale.split(" - ")[-1]
        auction_locale = re.sub(r'(\d)([A-Za-z])$', r'\1-\2', auction_locale)
        print(auction_locale)
        # clicking on the join button
        await join_button.click()
        complete_link = baseauction_url+auction_locale
        collection.find_one_and_update({"link": complete_link}, {"$set": {"Info": "Processing"}})
        collected_auctions.add(baseauction_url+auction_locale)

In [ ]:
# data={}
count=0
while True:
    count+=1
    all_ended_auctions=[i for i in all_auctions if await i.query_selector('div.sale-end.text-center')]
    if (len(all_auctions)==len(all_ended_auctions)) or (all_ended_auctions>6):
        print("No of auctions ended: ",len(all_ended_auctions))
        break

    for check in all_auctions:
        # For extracting data from auction
        car_link=await check.query_selector('a.titlelbl.ellipsis')
        try:
            link=await car_link.get_attribute("href")
            # print(link)
            if "https" in link:
                car_price=await check.query_selector('text.ng-star-inserted')
                checked=await car_price.text_content()
                price=checked.replace("\n","").replace(" ","")
                if "$" in price:
                    data[link]=price

            # Convert price to a number, assuming it's a string like "$1000"
            new_price = float(price.replace("$", "").replace(",","")) if price and "$" in price else 0
            if str(link) in data:
                # Get the existing price and convert it to a number
                existing_price = float(data[str(link)].replace("$", "").replace(",","")) if data[str(link)] else 0

                # Update the price only if the new price is greater than the existing one
                if new_price > existing_price:
                    data[str(link)] = price

                    # print({link: price}, end=' , ')
            else:
                # If the identity link is not in data, add it
                data[str(link)] = price
            # print({link: price}, end=' , ')
        except:
            pass
    if count>25:
        print(data)
        iframe_element=await page.query_selector('div.auction5iframe')
        iframe=await iframe_element.query_selector("iframe")
        content = await iframe.content_frame()
        all_auctions=await content.query_selector_all('gridster-item.ng-star-inserted')
        count=0

In [ ]:
await page.close()
await browser.close()

In [ ]:
from datetime import datetime
import pytz
cdt=pytz.timezone('America/Chicago')

data_list = [{"carLink": k, "price": v , "date": datetime.now(cdt).date().strftime("%d-%m-%Y").replace('-','.')} for k, v in data.items() if k != 'None' and v != ""]
carLink_list = [i['carLink'] for i in data_list]

subprocess.Popen(["python3", "check_link.py", ' '.join(carLink_list)])
print(f"Data captured of {len(data_list)} cars")
print(data_list)

from pymongo import UpdateOne
# Prepare bulk write operations
operations = []
for carsdata in data_list:
    price_obj = {"date": carsdata["date"], "price": carsdata["price"]}
    operations.append(
        UpdateOne(
            {"carLink": carsdata["carLink"]},
            {"$push": {"prices": price_obj}, "$setOnInsert": {"carLink": carsdata["carLink"]}},
            upsert=True
        )
    )

carsPrice_collection = db['CarsPrice']
# Execute bulk write operations
carsPrice_collection.bulk_write(operations)
# collection.insert_many(data_list)
        

In [ ]:
for auction in collected_auctions:
    collection.find_one_and_update({"link": auction}, {"$set": {"Info": "done"}})

In [ ]:
collected_auctions

In [ ]:
count=0

for document in cursor:
    link=document['link']
    if link not in instances:
        instances[link]=await context.new_page()
        await instances[link].goto(link)
        count+=1
        print(count)
    # break
    await asyncio.sleep(5)
    if count==5:
        break


In [ ]:
# while True:
#     for i in instances:  
#         try:
#             # car_link=await iframe_content.query_selector('a.titlelbl.ellipsis')
#             car_link=await instances[i].wait_for_selector('a.titlelbl.ellipsis')
#             link=await car_link.get_attribute("href")
#             if "https" in link:
#                 # car_price=await iframe_content.query_selector('text.ng-star-inserted')
#                 car_price=await instances[i].wait_for_selector('text.ng-star-inserted')
#                 checked=await car_price.text_content()
#                 checked=checked.replace("\n","").replace(" ","")
#                 if "$" in checked:
#                     data[link]=checked
#         except:
#             check=await instances[i].query_selector('div.sale-end.text-center')
#             if await check.text_content() == "Auction Ended":
#                 print("Auction Ended")
#                 # update the database
#                 collection.update_one({'link': i}, {'$set': {'Info': "Auction Ended"}})
#                 await instances[i].close()

In [ ]:
# NewPage = await context.new_page()
# auction_link=collection.find_one({'Info': "None"})["link"]
# print(auction_link)

In [ ]:
# for i in instances:
#     print(i)
#     await instances[i].close()
    # await asyncio.sleep(5)

In [ ]:
# await NewPage.goto(auction_link)

In [ ]:
for i in instances:
    iframe_element=await instances[i].query_selector('div.auction5iframe')
    elements[i]=await iframe_element.query_selector("iframe")
    elements[i] = await elements[i].content_frame()

In [ ]:
# <div _ngcontent-c10="" class="sale-end text-center">Auction Ended</div>
# for i in instances:
#     instances[i]=await instances[i].query_selector('div.auction5iframe')

In [ ]:
# html=await check.query_selector("iframe")
# iframe_content = await html.content_frame()

In [ ]:
data={}

In [ ]:
while elements:
    for i in elements:  
        try:
            # car_link=await iframe_content.query_selector('a.titlelbl.ellipsis')
            car_link=await elements[i].wait_for_selector('a.titlelbl.ellipsis')
            link=await car_link.get_attribute("href")
            if "https" in link:
                # car_price=await iframe_content.query_selector('text.ng-star-inserted')
                car_price=await elements[i].wait_for_selector('text.ng-star-inserted')
                checked=await car_price.text_content()
                checked=checked.replace("\n","").replace(" ","")
                if "$" in checked:
                    data[link]=checked
        except:
            check=await elements[i].query_selector('div.sale-end.text-center')
            if await check.text_content() == "Auction Ended":
                print("Auction Ended")
                await instances[i].close()
                del instances[i]
                del elements[i]
                
    # print(data)

In [ ]:
print(data)

In [ ]:
for i in elements:
    check=await elements[i].query_selector('div.sale-end.text-center')
    if await check.text_content() == "Auction Ended":
        print("Auction Ended")

In [ ]:
# while True :
#     car_link=await iframe_content.query_selector('a.titlelbl.ellipsis')
#     link=await car_link.get_attribute("href")
#     if "https" in link:
#         car_price=await iframe_content.query_selector('text.ng-star-inserted')
#         checked=await car_price.text_content()
#         checked=checked.replace("\n","").replace(" ","")
#         if "$" in checked:
#             data[link]=checked
    # print(data)